# 5.2 CNN trained with small dataset
In this notebook, we're gonna solve problem of classification dogs vs cats. Dataset has 4000 images, equal for both classes.

## 5.2.2 Load *Dogs vs Cats* dataset
We're gonna load it from Kaggle - it was challange in 2013.